<img src="img/docker-logo.png">
https://www.docker.com/

Docker

Docker - это самая популярная система контейнеризации приложений. По техническим причинам, у нас нет возможности запустить и продемонстрировать возможности Docker внутри Coursera Labs, так как они сами используют Docker для своей работы.

Поэтому для демонстрации возможностей докера, мы воспользуемся сервисом **play-with-docker.com** , который спонсируется самими разработчиками.
Этот сервис позволяет запустить виртуальное окружение на 4 часа с установленными инструментами докера.

# [Открыть Play With Docker](https://labs.play-with-docker.com/)

Для работы с этим сервисом, необходимо пройти регистрацию на официальном сайте docker.com. 

Ниже в ноутбуке будут команды, которые можно запускать внутри play with docker.

Первое что нужно сделать - это создать новый инстанс (виртуальну машину). Для текущих задач нам хватит ровно одной машины. В следующей лабораторной мы разберем, как управлять сразу кластером машин.

# Dockerfile

Создадим файл Dockerfile. В нем будем писать документацию по разработке.

```bash
touch Dockerfile
```

После создания файла, можно нажать Editor и добавить содержимое файла через графический интерфейс сайта.
Добавим туда самый простой докерфайл, который был на лекции.

```dockerfile
FROM ubuntu:16.04

ENTRYPOINT ["/bin/bash", "-c", "echo hello"]

```

Соберем этот контейнер и назовем его my-first-container

```bash
docker build -t my-first-container:latest .
```

Отлично, контейнер собрался!
Запустим его

```bash
docker run my-first-container:latest
```

Если все было сделано правильно, на экран должно было быть выведенно жизнерадостное hello.


# Контейнеризируем программу на Python

Попробуем собрать что-то более сложное - для этого нам потребуется Python.
По умолчанию его нет в образе. Чтобы он оказался у нас в контейнере можно или установить его самому или использовать базовый образ, в котором уже есть python.


1 - Устанавливаем самостоятельно
```dockerfile
FROM ubuntu:16.04

RUN apt-get -y update && apt-get install python3 -y

ENTRYPOINT ["python3", "-c", "print('hello from python')"]
```

2 - Используем уже готовый образ
```dockerfile
FROM python:3.7

ENTRYPOINT ["python3", "-c", "print('hello from python')"]
```

Сохраняем любой вариант в Dockerfile, собираем и проверяем работоспособность нашего контейнера.

```bash
docker build -t my-python-hello:latest .
docker run my-python-hello:latest
```

# Пакуем внешний скрипт в контейнер

Пора добавить в наш контейнер более сложную программу

Создадим application.py, добавим в него код, который бы выводил текущую директорию и ее содержимое. Этот файл после добавим в наш конейнер и запустим.


*application.py*
```python
import os

print("Right now I am here - {}".format(os.getcwd()))

content = os.listdir('.')
print("There are {} elements in this directory".format(len(content)))
for element in content:
    print(element)
```


Перед сборкой проверим, что сам файл успешно запускается
```bash
python3 application.py
```


Теперь соберем 

*Dockerfile*
```dockerfile
FROM python:3.7

COPY application.py application.py

ENTRYPOINT ["python3", "application.py"]
```

и запустим внутри контейнера


```bash
docker build -t python-ls:latest .
docker run python-ls:latest
```

Важно отметить, что вывод программы отличается! Так происходит, потому что приложение в контейнере изолировано от основной операционной системы и внутри имеет свою собственную файловую систему.

# Проброс параметров командной строки

Чтобы программа внутри контейнера могла быть полезной, необходимо научиться коммуницировать с ней извне.

Самый базовый способ - это проброс параметров. Все параметры, которые будут переданы после названия контейнера в команде `docker run` будут переданы внутри контейнера как обычные параметры.

Создадим программу на python, которая будет получать число и печатать слово Hello указанное количество раз.

*application.py*
```python
import sys

N = int(sys.argv[1])
for i in range(N):
    print("Hello")
   
```

Теперь соберем и попробуем запустить с параметрами
```bash
docker build -t python-hello:latest .
docker run python-hello:latest 3
docker run python-hello:latest 10
```

# Монтирование директории

Следующий способ взаимодействия с конейнером - это монтирование директории. Примонтированная директория становится общей для конейнера и основной операционной системы. Это позволяет передавать внутрь докера какие-то файлы для обработки, а также получать какие-то файловые результаты из докера.

Модифицируем нашу предыдущую программу, чтобы она показывала состояние директории `/sync-folder` , в которую мы будем монтировать различные директории извне.


*application.py*
```python
import os
import sys

target_dir = sys.argv[1]

print("Observe directory - {}".format(target_dir))

content = os.listdir(target_dir)
print("There are {} elements in this directory".format(len(content)))
for element in content:
    print(element)
```

Теперь соберем и попробуем запустить с примонтированной директорией. Для монтирования необходимо указать флаг `-v` в котором указать через двоеточие какие директории мы хотим засинхронизировать. Первая директория - из основной ОС, вторая - внутри контейнера.

```bash
docker build -t python-ls:latest .

docker run -v /root:/sync-folder python-ls:latest /sync-folder
docker run -v /:/sync-folder python-ls:latest /sync-folder
```

Если мы не укажем параметры монтирования, то такой директории не появится внутри контейнера и соответственно наша программа просто упадет, не найдя ее.

```bash
docker run python-ls:latest
```

# Проброс портов

Наиболее частый способ коммуницирования с контейнером - это проброс портов. Для этого указываются два порта - один для основной ОС, второй для контейнера. Все соединения с внешним портом будут пробрасываться внутрь контейнера. Для проброса необходим использовать ключ `-p` - формат точно такой же, как и у `-v`. 

Для демонстрации этой фукнции, воспользуемся встроенным в python модулем http.server, который поднимает файловый сервер в директории, из которой был запущен. Для начала просто проверим, как он работает.


Запустим
```bash
python3 -m http.server --bind 0.0.0.0 8000
```

Это должно запустить файловый сервер в текущей директории.

Чтобы проверить, что он работает, нажмем на `OPEN PORT` и укажем там 8000.

В браузере мы должны увидеть содержимое текущей директории.

Попробуем теперь упаковать это внутрь контейнера.


*Dockerfile*
```dockerfile
FROM python:3.7

WORKDIR /sync-folder

ENTRYPOINT ["python3", "-m", "http.server", "--bind", "0.0.0.0", "8080"]
```

Собираем и проверяем

```bash
docker build -t python-server:latest .
docker run -v /root:/sync-folder -p 9090:8080 python-server:latest
```

Открываем 9090 порт с помощью `OPEN PORT` и проверяем, что действительно работает

Схема того, как это работает:
<img src="img/python-server-schema.png">

Теперь этот контейнер можно использовать даже на тех машинах, на которых не установлен Python! Все зависимости лежат уже внутри контейнера и для запуска требуется лишь Docker.